# Write prompts using OPENAI API keys that perform the following task and display the output:

1. Write a chatbot prompt to iteratively create a sequence of chats on one particular topic.

    a. Ask the bot to solve one complex math problem.
    
    b. Give a PDF and website document; ask the bot to rewrite and answer questions on the given PDF and website.

    c. At the end, ask the bot to summarize your chat

In [1]:
! pip install openai==0.28
# ! pip install python-dotenv

  Obtaining dependency information for openai==0.28 from https://files.pythonhosted.org/packages/ae/59/911d6e5f1d7514d79c527067643376cddcf4cb8d1728e599b3b03ab51c69/openai-0.28.0-py3-none-any.whl.metadata
  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)


In [95]:
! source ~/.zshrc

In [102]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [26]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# a. Ask the bot to solve one complex math problem.

In [66]:
maths_questions = f"""
solve the equation 6x² + 11x - 35 = 0
"""

maths_questions_response = get_completion(maths_questions)
print(maths_questions_response)

To solve the quadratic equation 6x² + 11x - 35 = 0, we can use the quadratic formula:

x = (-b ± √(b² - 4ac)) / 2a

In this equation, a = 6, b = 11, and c = -35.

Plugging these values into the formula, we get:

x = (-11 ± √(11² - 4*6*(-35))) / 2*6
x = (-11 ± √(121 + 840)) / 12
x = (-11 ± √961) / 12
x = (-11 ± 31) / 12

Now we have two possible solutions:

x₁ = (-11 + 31) / 12 = 20 / 12 = 5/3
x₂ = (-11 - 31) / 12 = -42 / 12 = -7/2

Therefore, the solutions to the equation 6x² + 11x - 35 = 0 are x = 5/3 and x = -7/2.


# b. Give a PDF and website document; ask the bot to rewrite and answer questions on the given PDF and website.

In [37]:
! pip install PyPDF2

  Obtaining dependency information for PyPDF2 from https://files.pythonhosted.org/packages/8e/5e/c86a5643653825d3c913719e788e41386bee415c2b87b4f955432f2de6b2/pypdf2-3.0.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:004.7 MB/s eta 0:00:01


In [44]:
import requests
from PyPDF2 import PdfReader


def extract_pdf_text(pdf_url):
    response = requests.get(pdf_url)
    with open('temp.pdf', 'wb') as f:
        f.write(response.content)
    
    pdf_text = ''
    with open('temp.pdf', 'rb') as pdf_file:
        reader = PdfReader(pdf_file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            pdf_text += page.extract_text()
    
    return pdf_text


extract_pdf_text("https://www.dcs.bbk.ac.uk/~dell/teaching/cc/book/databricks/spark-intro.pdf")[:100]

'A Gentle Introduction to2Apache Spark has seen immense growth over the past \nseveral years. The size'

In [45]:
from bs4 import BeautifulSoup
    
def extract_website_text(website_url):    
    response = requests.get(website_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    website_text = '\n'.join([p.get_text() for p in soup.find_all('p')])
    return website_text


extract_website_text("https://medium.com/@vdwayne/yosemite-national-park-a-natural-marvel-with-a-ton-of-wasted-space-626b1f185df6")[:100]

'Sign up\nSign in\nSign up\nSign in\nvdwayne\nFollow\n--\nListen\nShare\nNestled within the Sierra Nevada Moun'

In [67]:
document_questions  = f"""
1 - Where is Yosemite located?.
2 - What are some iconic places in Yosemite National Park?
3 - What is spark ?
4 - How spark works ?
"""

pdf_data = extract_pdf_text("https://www.dcs.bbk.ac.uk/~dell/teaching/cc/book/databricks/spark-intro.pdf")[:10000]
website_data = extract_website_text("https://medium.com/@vdwayne/yosemite-national-park-a-natural-marvel-with-a-ton-of-wasted-space-626b1f185df6")[:5000]

document_questions_prompt = f"""
Answer following questions from the given Text: 
{questions}

Separate your answers with line breaks.

Text:
```{pdf_data}```

```{website_data}```
"""
document_questions_response = get_completion(document_questions_prompt)


In [85]:
questions = [question for question in document_questions.splitlines() if len(question.strip()) > 0]
answers = [answer for answer in document_questions_response.split("\n") if len(answer.strip()) > 0]

for q, a in zip(questions, answers):
    print(f"""
Question: {q.split(" - ")[1]}
Answer  : {a}
""")


Question: Where is Yosemite located?.
Answer  : 1 - Where is Yosemite located?.


Question: What are some iconic places in Yosemite National Park?
Answer  : Yosemite National Park is located within the Sierra Nevada Mountains of California.


Question: What is spark ?
Answer  : 2 - What are some iconic places in Yosemite National Park?


Question: How spark works ?
Answer  : Some iconic places in Yosemite National Park include El Capitan, Half Dome, Bridalveil Fall, Yosemite Falls, Vernal and Nevada Falls, and Ribbon Fall.



# c. At the end, ask the bot to summarize your chat

In [91]:
summary_question = f"""

Summarize all the above chats I did with the openai apis. Here are all the interations in details having one line for each question and summzarize answers. Make the overall reponse 500 characters long. Give the reponse in differnet points seperated by line breaks

```Question - {maths_questions}```
```Bot Answer - {maths_questions_response}```

```Question - {document_questions_prompt}```
```Bot Answer - {document_questions_response}```

"""
response = get_completion(summary_question)
response.split("\n")


['The quadratic equation 6x² + 11x - 35 = 0 has solutions x = 5/3 and x = -7/2.',
 'Yosemite National Park is located in the Sierra Nevada Mountains of California and features iconic landmarks like El Capitan and Half Dome.',
 'Spark is a unified computing engine and library for parallel data processing on computer clusters.',
 'Spark works by providing a unified platform for big data applications, supporting various tasks from SQL to machine learning, and running on computer clusters for scalable processing.']